In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-05-31 23:13:48--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-05-31 23:13:49 (56.3 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [10]:
import minsearch
import json

In [12]:
with open('documents.json', 'rt') as f_in:
    documents_raw = json.load(f_in)

In [13]:
documents = []

for course_dict in documents_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [14]:
import openai
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def search(query):

    index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
    )

    index.fit(documents)

    search_results = index.search(
        query=query,
        num_results=5
        )
    
    return search_results

In [48]:
def build_prompt(query, search_results):

    prompt_template = """
    You are a helpful teaching assistant. Answer the given QUESTION from the provided CONTEXT. If the answer is not in the provided CONTEXT then say so.

    QUESTION: {question}

    CONTEXT: {context}

    ANSWER:

    """

    context = ""

    for i in search_results:
        context = context + f"section: {i['section']}\nquestion: {i['question']}\nanswer: {i['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context)

    return prompt

In [49]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user','content':prompt}]
    )
    return response.choices[0].message.content.strip()

In [50]:
query = 'how do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    response = llm(prompt)
    return response

In [51]:
rag(query)

'The answer is not in the provided CONTEXT.'

In [55]:
print(llm(build_prompt(query,search(query))))

The answer is not in the provided CONTEXT.
